## The method of stock prediction with various analize method (For practice)
- This is my python project for improving my skill
- For stock prediction, the proposed method uses various algorithms such as statistical analysis, probability model and neural networks
- The simple s&p500 data is used


## 다양한 분석 기법을 활용한 주가 예측 방법 (연습) 
- 파이썬을 활용하여 다양한 분석 기법을 활용한 주가 예측 방법 구현 수행 (습작용)
- 그냥 널리 알려져 있는 통계적 분석, 확률모델, 인공 신경망 등등 여러 방법 활용
- 데이터 역시 단순한 종가 등 수학적 정보만 활용 
- 어떤 분석 방법이 예측력이 좋은지, 어떻게 하면 예측력을 높일 수 있는지 등


## 고려사항 
- y 값의 변화 
    - 기본적으로 학습을 위해 활용하는 y의 값이 지수값을 사용하는 경우, 성능이 떨어진다(경험상) 
    - y값을 결과적으로 up/down 등 라벨을 예측하는 쪽의 더 현실성 있다
    - 다양한 형태의 y 값의 변화 및 성능비교 수행 
- 지수 값 활용시 수익률로 변화 
    - 시간이 지나면서 규모 자체가 바뀔 수 있음 (10년전의 짜장면 값과 지금 짜장면 값의 차이 존재) 
    - 전 시간 영역에서 학습을 하는 것이기 때문에 스케일 조절 필요
        - 수익률 (미래 - 현재)/현재 , (현재 - 과거)/과거 



---
## 시도해볼것들 
### 토픽 모델링 
### 인공 신경망 
### 평가 방법 
### 비교 평가 
--- 
## Reference 
- http://3months.tistory.com/137 
    - 파이썬으로 선형 회귀 분석
- http://uncoded.tistory.com/21 
    - 파이썬으로 로지스틱 회귀 분석 구현 
---
    

In [160]:
''' 
Library  
'''
# default 
import numpy as np 
import pandas as pd
from pprint import pprint

# For regression   
from sklearn import linear_model 
import matplotlib.pyplot as plt 



## 1) Data read and describe part 
- using pandas, read csv file 
---
- pandas 라이브러리를 활용하여 stock data(csv)를 읽어오자 
- 데이터에 대한 성격을 보자 
- 데이터 분리 622 / 82 


In [60]:
'''
date = 날짜 
open = 시가? - 시작 시 가격  
high = 고가 - 당일 최고가격 
low = 저가 - 당일 최저가격 
close = 종가 - 장 마감 가격 
volume = 거래량 
Name = sp500 회사이름 - 약어 
'''
temp_sp500_data = pd.read_csv('all_stocks_5yr.csv', header= 0, names=['date','open','high','low','close','volume','Name'])
temp_sp500_data.describe()

,open,high,low,close,volume
count,619029.000000,619032.000000,619032.000000,619040.000000,6.190400e+05
mean,83.023334,83.778311,82.256096,83.043763,4.321823e+06
std,97.378769,98.207519,96.507421,97.389748,8.693610e+06
min,1.620000,1.690000,1.500000,1.590000,0.000000e+00
25%,40.220000,40.620000,39.830000,40.245000,1.070320e+06
50%,62.590000,63.150000,62.020000,62.620000,2.082094e+06
75%,94.370000,95.180000,93.540000,94.410000,4.284509e+06
max,2044.000000,2067.990000,2035.110000,2049.000000,6.182376e+08


In [186]:
sp500_data = temp_sp500_data
sp500_data = pd.concat([sp500_data, sp500_data['close'].shift(1).rename('lag1'), 
                        sp500_data['close'].shift(2).rename('lag2'), 
                        sp500_data['close'].shift(3).rename('lag3'),
                        sp500_data['close'].shift(4).rename('lag4'), 
                        sp500_data['close'].shift(5).rename('lag5'),
                        sp500_data['close'].shift(6).rename('lag6'), 
                        sp500_data['close'].shift(7).rename('lag7'), 
                        sp500_data['close'].shift(-1).rename('fu1'),
                       sp500_data['close'].shift(-7).rename('fu7'),
                       sp500_data['close'].shift(-30).rename('fu30')], 
                        axis=1).dropna()

In [188]:
sp500_data['lag1'] = (sp500_data['close']-sp500_data['lag1'])/sp500_data['lag1']
sp500_data['lag2'] = (sp500_data['close']-sp500_data['lag2'])/sp500_data['lag2']
sp500_data['lag3'] = (sp500_data['close']-sp500_data['lag3'])/sp500_data['lag3']
sp500_data['lag4'] = (sp500_data['close']-sp500_data['lag4'])/sp500_data['lag4']
sp500_data['lag5'] = (sp500_data['close']-sp500_data['lag5'])/sp500_data['lag5']
sp500_data['lag6'] = (sp500_data['close']-sp500_data['lag6'])/sp500_data['lag6']
sp500_data['lag7'] = (sp500_data['close']-sp500_data['lag7'])/sp500_data['lag7']
sp500_data['fu1'] = (sp500_data['fu1']-sp500_data['close'])/sp500_data['close']
sp500_data['fu7'] = (sp500_data['fu7']-sp500_data['close'])/sp500_data['close']
sp500_data['fu30'] = (sp500_data['fu30']-sp500_data['close'])/sp500_data['close']

In [189]:
# data 확인 
sp500_data[:2]

,date,open,high,low,close,volume,Name,lag1,lag2,lag3,lag4,lag5,lag6,lag7,fu1,fu7,fu30
7,2013-02-20,14.17,14.26,13.15,13.33,14725200,AAL,-0.065217,-0.080690,-0.047177,-0.090723,-0.065872,-0.078147,-0.096271,0.003001,0.021005,0.177044
8,2013-02-21,13.62,13.95,12.90,13.37,11922100,AAL,0.003001,-0.062412,-0.077931,-0.044317,-0.087995,-0.063069,-0.075380,0.014959,0.039641,0.175767


In [190]:
print('A number of companies in sp500 data is ', len(list(set(sp500_data['Name']))) ) 

A number of companies in sp500 data is  505


In [191]:
#split data 
print(len(sp500_data))
_622_data = sp500_data[0:int(len(sp500_data)*0.6)]
_622_data_eval = sp500_data[int(len(sp500_data)*0.6)+1:int(len(sp500_data)*0.8)]
_622_data_test = sp500_data[int(len(sp500_data)*0.8)+1:-1]

_82_data = sp500_data[0:int(len(sp500_data)*0.8)]
_82_data_eval = sp500_data[int(len(sp500_data)*0.8)+1:-1]




618992


In [192]:
_622_data[:3]

,date,open,high,low,close,volume,Name,lag1,lag2,lag3,lag4,lag5,lag6,lag7,fu1,fu7,fu30
7,2013-02-20,14.17,14.26,13.15,13.33,14725200,AAL,-0.065217,-0.080690,-0.047177,-0.090723,-0.065872,-0.078147,-0.096271,0.003001,0.021005,0.177044
8,2013-02-21,13.62,13.95,12.90,13.37,11922100,AAL,0.003001,-0.062412,-0.077931,-0.044317,-0.087995,-0.063069,-0.075380,0.014959,0.039641,0.175767
9,2013-02-22,13.57,13.60,13.21,13.57,6071400,AAL,0.014959,0.018005,-0.048387,-0.064138,-0.030021,-0.074352,-0.049054,-0.040531,0.035372,0.167281



## 2) 회귀모델 적용(regression model) 
- 선형 회귀 모델을 이용한 지수 예측 
- 로지스틱 회귀 모델을 이용한 up/down 예측 
- 기타 

### [1] 단순 선형 회귀 모형 
- 단순 선형 회귀 모델로 Y(결과)에 대한 X(정보)를 fitting한다 식은 아래와 같다
- 결과물은 model.coef (정보에 대한 가중치)와 model.intercept(절편)에 저장된다 
$$ Y \sim  + W_{1}*lag1+...+ W_{7}*lag7 + intercept $$ 



In [238]:
#basic linear regression 
model = linear_model.LinearRegression()
x_vars1 = ['lag1','lag2','lag3','lag4','lag5','lag6','lag7']
model_1_1 = model.fit(_622_data[x_vars1], _622_data["fu1"]) # model.fit(x,y)
model = linear_model.LinearRegression()
model_1_7 = model.fit(_622_data[x_vars1], _622_data["fu7"]) # model.fit(x,y)
model = linear_model.LinearRegression()
model_1_30 = model.fit(_622_data[x_vars1], _622_data["fu30"]) # model.fit(x,y)
print("가중치 = ", model_1_1.coef_, "\n절편 =", model_1_1.intercept_)
print("가중치 = ", model_1_7.coef_, "\n절편 =", model_1_7.intercept_)
print("가중치 = ", model_1_30.coef_, "\n절편 =", model_1_30.intercept_)


가중치 =  [-0.00338599  0.00205511  0.00059207 -0.00140981 -0.00137586  0.00114218
 -0.00125336] 
절편 = 0.0005793261728476748
가중치 =  [-0.00265725  0.00194295 -0.00079439 -0.00175215 -0.00081738  0.00018786
 -0.00762894] 
절편 = 0.0039592349840759464
가중치 =  [-2.78721671e-03 -2.65096575e-05 -2.82719242e-03 -3.62032712e-03
 -2.35453682e-03 -1.42886675e-03 -1.57456474e-02] 
절편 = 0.016296434565986388


### [2] 교호작용을 고려한 모델 
- 교호작용은 effect modification이라고 불리며, 한 변수에 의해 다른 변수의 효과가 변화하는 모델로 아래의 수식과 같음  
$$ Y \sim W_{1}*close + W_{2}*lag1+...+ W_{n}*(diff1* volume/max\;volume) + intercept $$ 
- 기존의 테이블에서 diff1 = (close - lag1)/lag1 * volume/max 을 수행함

In [239]:
_622_data = _622_data.assign(clo_vol=(_622_data['lag1'])*_622_data['volume']/_622_data['volume'].max())
_622_data_eval = _622_data_eval.assign(clo_vol=(-_622_data_eval['lag1'])*_622_data_eval['volume']/_622_data_eval['volume'].max())
_622_data_test = _622_data_test.assign(clo_vol=(-_622_data_test['lag1'])*_622_data_test['volume']/_622_data_test['volume'].max())

In [240]:
model2 = linear_model.LinearRegression()
x_vars2 = ['lag1','lag2','lag3','lag4','lag5','lag6','lag7','clo_vol']
model_2_1 = model2.fit(_622_data[x_vars2], _622_data["fu1"]) # model.fit(x,y)
model2 = linear_model.LinearRegression()
model_2_7 = model2.fit(_622_data[x_vars2], _622_data["fu7"]) # model.fit(x,y)
model2 = linear_model.LinearRegression()
model_2_30 = model2.fit(_622_data[x_vars2], _622_data["fu30"]) # model.fit(x,y)

print("가중치 = ", model_2_1.coef_, "\n절편 =", model_2_1.intercept_)

가중치 =  [-1.28802099e-04  2.15992405e-03  5.56489240e-04 -1.40130141e-03
 -1.32090914e-03  1.12658684e-03 -1.23585413e-03 -3.41152562e-01] 
절편 = 0.0005775203370836131


### [3] 평가하기 
- 1일과 30일 에측 관련하여 MAE와 MSE 값을 도출하였다 
- "model 1"은 lag의 수익률을 기반으로 피팅한 것이고 "model 2"는 lag의 수익률과 더불어 현재 볼륨을 이용하였다 



In [241]:

print("The MSE of model 1 for 1 day futuer is %.10f"% np.mean((model_1_1.predict(_622_data_test[x_vars1]) - _622_data_test['fu1']) ** 2))
print("The MAE of model 1 for 1 day futuer is %.10f"% np.mean(abs(model_1_1.predict(_622_data_test[x_vars1]) - _622_data_test['fu1'])))

print("The MSE of model 2 for 1 day futuer is %.10f"% np.mean((model_2_1.predict(_622_data_test[x_vars2]) - _622_data_test['fu1']) ** 2))
print("The MAE of model 2 for 1 day futuer is %.10f"% np.mean(abs(model_2_1.predict(_622_data_test[x_vars2]) - _622_data_test['fu1']) ))

The MSE of model 1 for 1 day futuer is 0.0008195663
The MAE of model 1 for 1 day futuer is 0.0107728672
The MSE of model 2 for 1 day futuer is 0.0008195743
The MAE of model 2 for 1 day futuer is 0.0107749589


In [242]:

print("The MSE of model 1 for 1 month futuer is %.10f"% np.mean((model_1_30.predict(_622_data_test[x_vars1]) - _622_data_test['fu30']) ** 2))
print("The MAE of model 1 for 1 month futuer is %.10f"% np.mean(abs(model_1_30.predict(_622_data_test[x_vars1]) - _622_data_test['fu30'])))


print("The MSE of model 2 for 1 month futuer is %.10f"% np.mean((model_2_30.predict(_622_data_test[x_vars2]) - _622_data_test['fu30']) ** 2))
print("The MAE of model 2 for 1 month futuer is %.10f"% np.mean(abs(model_2_30.predict(_622_data_test[x_vars2]) - _622_data_test['fu30']) ))

The MSE of model 1 for 1 month futuer is 0.0220214052
The MAE of model 1 for 1 month futuer is 0.0717871438
The MSE of model 2 for 1 month futuer is 0.0220229661
The MAE of model 2 for 1 month futuer is 0.0717905728


### 수행한 내역 정리 및 future work 
- 회귀 분석 관련되어 작성 중 
- 일단 회귀 분석하여 튜닝하는 작업 수행 
- 82 데이텅